# *pathways*. Global sensitivity analysis application

This notebook contains the code necessary to generate the results shown in the publication *"Exploring uncertainties in the material-energy nexus for net zero transition scenarios"* 

Data packages were generated using *premise*, and can be found in this folder.

Contact:

- romain.sacchi@psi.ch
- alvaro.hahn-menacho@psi.ch

## Import *pathways* and other necessary modules

In [1]:
from pathways import Pathways, run_gsa

In [2]:
import numpy as np
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path

## Set calculation parameters and export as .gzip

In [3]:
p = Pathways(
    datapackage="remind-SSP2-PkBudg1150-stem-SPS1.zip",
    geography_mapping="geo_mapping_remind.yaml",
    activities_mapping="act_categories_agg_unique.yaml",
)

Invalid datapackage: Descriptor validation error: {'path': 'mapping/mapping.yaml', 'profile': 'data-resource', 'name': 'mapping', 'format': 'yaml', 'mediatype': 'text/yaml', 'encoding': 'utf-8'} is not valid under any of the given schemas at "resources/29" in descriptor and at "properties/resources/items/oneOf" in profile
Invalid datapackage: Descriptor validation error: 'data-resource' is not one of ['tabular-data-resource'] at "resources/29/profile" in descriptor and at "properties/resources/items/properties/profile/enum" in profile


In [4]:
# vars = ['FE_cars_electricity','FE_cars_gas', 'FE_cars_hydrogen', 'FE_cars_diesel','FE_cars_gasoline']

In [5]:
vars = [v for v in p.scenarios.coords["variables"].values if v.startswith("FE")]

In [ ]:
p.calculate(
    methods=[
        'EF v3.1 EN15804 - climate change - global warming potential (GWP100)',
        'EF v3.1 EN15804 - acidification - accumulated exceedance (AE)',
        'EF v3.1 EN15804 - ecotoxicity: freshwater - comparative toxic unit for ecosystems (CTUe)',
        'EF v3.1 EN15804 - material resources: metals/minerals - abiotic depletion potential (ADP): elements (ultimate reserves)',
        'EF v3.1 EN15804 - eutrophication: freshwater - fraction of nutrients reaching freshwater end compartment (P)',
        'EF v3.1 EN15804 - photochemical oxidant formation: human health - tropospheric ozone concentration increase',
        'Inventory results and indicators - resources - total freshwater extraction',
        'Crustal Scarcity Indicator 2020 - material resources: metals/minerals - crustal scarcity potential (CSP)',
        'Inventory results and indicators - resources - total surface occupation',
        # 'RELICS - metals extraction - Lithium',
        # 'RELICS - metals extraction - Vanadium',
        # 'RELICS - metals extraction - Copper',
        # 'RELICS - metals extraction - Platinum',
        # 'RELICS - metals extraction - Phosphorus',
        # 'RELICS - metals extraction - Lead',
        # 'RELICS - metals extraction - Gallium',
        # 'RELICS - metals extraction - Arsenic',
        
        'Inventory results and indicators - resources - land occupation',
        # 'Inventory results and indicators - resources - total water extraction',
        # 'Inventory results and indicators - resources - total freshwater extraction',
        # 'ReCiPe 2016 v1.03, midpoint (H) - water use - water consumption potential (WCP)',
        'ReCiPe 2016 v1.03, midpoint (H) - particulate matter formation - particulate matter formation potential (PMFP)',
    ] + [m for m in p.lcia_methods if "RELICS" in m],
    regions=["CH"],
    scenarios=p.scenarios.pathway.values.tolist(),
    years=[2020, 2030, 2040, 2050],
    variables=vars,
    use_distributions=1000,
    subshares=True,
    multiprocessing=True,
)

Calculating LCA results for remind...
--- Calculating LCA results for SSP2-PkBudg1150-SPS1...


In [ ]:
p.export_results()

In [ ]:
run_gsa()

In [6]:
fp = p.export_results()

# interpolate in-between years
import pandas as pd

df = pd.read_parquet(fp, engine='pyarrow')

df = df[df["value"]!=0.0]
df = df[~df["value"].isnull()]
print(len(df))
df=df.reset_index()

# pivottablejs is very convenient way to visualize pivot tables
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='example.html')

Results exported to results_20240801_180657.gzip
622528


In [1]:
import yaml

import logging
import numpy as np
from collections import defaultdict

def check_subshares(data: dict) -> dict:
    """
    Adjusts the values in 'data' for each year ensuring the sum equals 1
    for each category, excluding technologies without a name.
    It dynamically identifies years (integer keys) that contain a 'value'
    subkey and adjusts them.

    :param data: A dictionary with categories as keys, each category is
    a dictionary of subcategories containing a list of technology dictionaries.
    :return: A dictionary with the adjusted values.
    """

    for category, technologies in data.items():
        technologies_to_remove = []
        totals = defaultdict(float)
        for technology, params in technologies.items():
            name = params.get("name")
            if name in {"null", "Null", None} or not name.strip():
                logging.warning(
                    f"Technology '{technology}' in category '{category}' is being removed due to invalid name '{name}'."
                )
                technologies_to_remove.append(technology)
                continue
            if "share" in params:
                for year, share in params["share"].items():
                    if "loc" in share:
                        totals[year] += share["loc"]
            else:
                logging.warning(
                    f"Technology '{technology}' in category '{category}' does not have a 'share' key"
                )

        for tech in technologies_to_remove:
            del technologies[tech]

        for year, total_value in totals.items():
            if not np.isclose(total_value, 1.00, rtol=1e-3):
                logging.warning(
                    f"Total of '{year}' values in category '{category}' does not add up to 1.00 (Total: {total_value}). Adjusting values."
                )
                for technology, params in technologies.items():
                    if "share" in params and year in params["share"] and "loc" in params["share"][year]:
                        # Adjust the share value
                        params["share"][year]["loc"] /= total_value

    return data

In [2]:
file_path = 'C:/Users/hahnme_a/PycharmProjects/pathways/pathways/data/technologies_shares.yaml'

In [3]:
with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

# Check and adjust the subshares
adjusted_data = check_subshares(data)

# Display the adjusted data for verification
adjusted_data

{'Wind': {'onshore-DDPM': {'name': 'electricity production, wind, 1-3MW turbine, onshore, direct drive',
   'reference product': 'electricity, high voltage',
   'unit': 'kilowatt hour',
   'share': {2020: {'loc': 0.19},
    2050: {'minimum': 0.17, 'maximum': 0.19, 'uncertainty_type': 'uniform'}}},
  'onshore-Gearbox': {'name': 'electricity production, wind, 1-3MW turbine, onshore',
   'reference product': 'electricity, high voltage',
   'unit': 'kilowatt hour',
   'share': {2020: {'loc': 0.77},
    2050: {'minimum': 0.6, 'maximum': 0.77, 'uncertainty_type': 'uniform'}}},
  'offshore-DDPM': {'name': 'electricity production, wind, 1-3MW turbine, offshore, direct drive',
   'reference product': 'electricity, high voltage',
   'unit': 'kilowatt hour',
   'share': {2020: {'loc': 0.022},
    2050: {'minimum': 0.022, 'maximum': 0.15, 'uncertainty_type': 'uniform'}}},
  'offshore-Gearbox': {'name': 'electricity production, wind, 1-3MW turbine, offshore',
   'reference product': 'electricity, h